# 7.6 Reduced Port Ball Valve

Water is discharged at 15 degrees Celsius from a tank with 7 m of head to atmosphere through:

* 60 meters of 80 mm schedule 40 pipe
* Six 80 mm standard 90 degree threaded elbows
* One 80 mm flanged ball valve, with a 60 mm diameter seat, 16 degree conical inlet and 30 degree conical outlet.
* The entrance is sharp-edged and flush with the tank

In [1]:
from thermo.units import Chemical
from fluids.units import *
from math import pi
water = Chemical('water', T=15*u.degC)
rho = water.rho
mu = water.mu

H = 7*u.m
L = 60*u.m
fd = 0.017 # assumed in their example
NPS, D_pipe, Do_pipe, t = nearest_pipe(Do=80*u.mm)

K = K_from_f(fd=fd, L=L, D=D_pipe)
K += entrance_sharp()
K += exit_normal()
K += 6*bend_rounded(D_pipe, angle=90*u.degrees, fd=fd, bend_diameters=0.65)
ball_valve_angle = 0.5*(15+30)*u.degrees # use the average angle
K += K_ball_valve_Crane(D1=D_pipe, D2=60*u.mm, angle=ball_valve_angle, fd=fd)

v = (2*u.gravity*H/K)**0.5
print('Velocity = %s' %v.to_base_units())
Q = v*pi/4*D_pipe**2
print('Flow rate = %s' %Q.to(u.L/u.min))
Re = Reynolds(D=D_pipe, rho=rho, mu=mu, V=v)
fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)

Velocity = 2.770041323755228 meter / second
Flow rate = 792.5474399133158 liter / minute


The radius of curvature of the elbows was not specified; 0.65 bend diameters matches their results most closely. They modified the ball valve equation to support both an inlet and an outlet angle; the average value is used here.

Their calculated values are 2.74 m/s and flow rate of 781 L/min.

The calculation can be performed more accurately by iterating; a naive approach is shown below. A very different flow rate is obtained when the roughness of the pipe is considered in the friction factor calculation.

In [2]:
fd = 0.017
for i in range(7):
    K = K_from_f(fd=fd, L=L, D=D_pipe)
    K += entrance_sharp()
    K += exit_normal()
    K += 6*bend_rounded(D_pipe, angle=90*u.degrees, fd=fd, bend_diameters=0.65)
    ball_valve_angle = 0.5*(15+30)*u.degrees # use the average angle
    K += K_ball_valve_Crane(D1=D_pipe, D2=60*u.mm, angle=ball_valve_angle, fd=fd)

    v = (2*u.gravity*H/K)**0.5
    Q = v*pi/4*D_pipe**2
    print('Flow rate = %s' %Q.to(u.L/u.min))
    Re = Reynolds(D=D_pipe, rho=rho, mu=mu, V=v)
    fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)

Flow rate = 792.5474399133158 liter / minute
Flow rate = 748.2664116002655 liter / minute
Flow rate = 746.5628913428789 liter / minute
Flow rate = 746.4941928055326 liter / minute
Flow rate = 746.4914172120895 liter / minute
Flow rate = 746.491305062719 liter / minute
Flow rate = 746.4913005312487 liter / minute
